In [1]:
from app import db, app
from shapely.geometry import shape


In [4]:
with app.app_context():
    db.drop_all()
    db.create_all()

In [4]:
from geoalchemy2.functions import ST_GeomFromGeoJSON
import geojson
from app.models import Shapefile
import os
import yaml

toronto = yaml.safe_load(open('localities.yml'))['toronto']

os.environ.get('WHOSONFIRST_PATH')


def get_path_from_id(id):
    s = str(id)
    id_path = '/data/' + s[:3] + '/' + s[3:6] + '/' + s[6:9]
    if len(s) > 9:  id_path += '/' + s[9:]
    return os.environ.get('WHOSONFIRST_PATH') + id_path + '/' + s + '.geojson'


get_path_from_id(toronto)
with open(get_path_from_id(toronto)) as f:
    gj = geojson.load(f)
    bbox = gj.bbox
    properties = gj.properties
    geometry = shape(gj.geometry).wkt
    id = gj.id

shape = Shapefile(id=id, geometry=geometry, bbox=bbox)



In [3]:
type(shape.geometry)


In [5]:

with app.app_context():
    db.session.add(shape)
    db.session.commit()

In [6]:
shape.bbox